# Laser Circles 

# Code:

In [15]:
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL
from utils import picture

sys.path.append("..")

In [ ]:
cnc.config


In [16]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
sleep(0.5)
cnc.laser_mode = 1
sleep(0.5)
print("Laser Mode: {}".format(cnc.laser_mode))


ok
Laser Mode: 1.0


In [17]:
cnc.cmd("?")


['ok', '<Idle|MPos:0.023,15.798,0.000|Bf:15,127|FS:0,0>', 'ok']

In [31]:
def init(feed=200, zero=True):
    program = GCode.GCode()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    if zero:
        program.G92(X=0, Y=0, Z=0)  # Zero position.
    # Set the laser on rates.
    program.G1(F=feed)
    program.G2(F=feed)
    program.G3(F=feed)
    return program


def end():
    program = GCode.GCode()
    program.G0(X=0, Y=0, Z=0)
    program.M5()
    return program


In [44]:
def circle(center_x=5, center_y=5, radius=5, laser_pwm=255):
    prog = GCode.GCode()
    prog.G0(X=center_x - radius, Y=center_y)
    prog.M3(S=laser_pwm)
    prog.G2(X=center_x - radius, Y=center_y, I=radius, J=0)
    prog.M5()
    prog.G0(X=center_x - radius, Y=center_y)
    return prog


In [66]:
cnc.run(["G0", "M3S1", "G1F1"])


0.3106093406677246

In [24]:
radius = 15


In [25]:
cnc.run(init(feed=200))


0.41327452659606934

In [26]:
cnc.run("G0X15Y15")


0.10416984558105469

In [41]:
cnc.run("G0X0Y15")


0.10398340225219727

In [34]:
cnc.run("M3S1")


0.10330462455749512

In [37]:
cnc.cmd("G2X15Y15R15F200")


['ok', 'error:33']

In [47]:
cnc.run(circle(15, 15, 15, 2))


29.354402542114258

In [49]:
cnc.run("G0X0Y0")


0.10359907150268555

In [67]:
X_centers = np.linspace(radius, 180 - radius, 5)
Y_centers = np.linspace(radius, 90 - radius, 4)
test_run = GCode.GCode()
test_run += init(feed=200)
for x_center in X_centers:
    for y_center in Y_centers:
        test_run += circle(
            center_x=x_center, center_y=y_center, radius=radius, laser_pwm=150
        )
test_run += end()


In [70]:
gcode_file = "CircleTests.gcode"
test_run.save(gcode_file)
del test_run
test_run = GCode.GCode()
test_run.load(gcode_file)
test_run


<GCode>[cmds=108]

In [71]:
from time import sleep

In [72]:
try:
    cnc.run(test_run)
    while "Idle" not in cnc.status:
        print(cnc.status)
        sleep(5)
except KeyboardInterrupt as error:
    print("Feed Hold")
    cnc.cmd("!")
    while 1:
        try:
            cnc.reset()
            break
        except:
            pass
    print("^C")


<Run|MPos:178.353,119.705,0.000|Bf:14,127|FS:235,0|Ov:100,100,100>
<Run|MPos:114.724,87.890,0.000|Bf:14,127|FS:894,0>
<Run|MPos:45.938,53.498,0.000|Bf:14,127|FS:894,0>


In [ ]:
cnc.cmd("G0X0Y0")


In [ ]:
cnc.reset()


In [ ]:
cnc.reset()


In [ ]:
picture()


# Test Aborted.

Cuts were way too aggressive.

False